In [3]:
'''READ ME: 

This is a Python Jupyter Notebook designed to run the classification, vectorization, and Excel sheet creation of the 
Variable Milling process. Please enter the necessary user inputs below and run the script as needed.

Best practice as always to bring all of the files locally somehwere on the C: Drive for the best performance.

Sebastian Brauer - 2024/06/04
'''

import arcpy
from datetime import datetime
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

arcpy.env.workspace = r'C:\GIS_Scripting\Geodatabases\Variable Milling Surfaces.gdb' # DO NOT CHANGE
arcpy.env.overwriteOutput = True

input_project_name = '_Willmar_TH_23' # REQUIRED USER INPUT always start with an underscore and then come up with a name for the project. The goal is to come up with a unique name, the city and TH make a good combo, feel free to add your last name as well. 
input_county_name = 'Kandiyohi'# REQUIRED USER INPUT in order to get the right projection. Please capitalize the first letter of the county, feel free to review the dictionary if needed.
input_multipatch_path = r'C:\Users\lschneider\Desktop\2024-07-25 Exported Delta Terrain.dgn\Multipatch' # REQUIRED USER INPUT in order to find the folder with the data needed for the analysis
input_numbers_rows = 14 # REQUIRED USER INPUT in order to make sure that there are the right number of reclasses when calculating the GRIDCODE.
csv_save_location = r'C:\Users\lschneider\OneDrive - Isthmus Engineering\Documents\Variable Milling TH 23.csv' # REQUIRED USER INPUT save location of the output csv

# Here is a dictionary that relates every county to a spatial projection number.

mn_county_systems_epsg = {
    "Aitkin": 103700,
    "Anoka": 103708,
    "Becker": 103709,
    "Beltrami North": 103710,
    "Beltrami South": 103711,
    "Benton": 103712,
    "Big Stone": 103713,
    "Blue Earth": 103714,
    "Brown": 103715,
    "Carlton": 103716,
    "Carver": 103717,
    "Cass North": 103718,
    "Cass South": 103719,
    "Chippewa": 103720,
    "Chisago": 103721,
    "Clay": 103701,
    "Clearwater": 103702,
    "Cook North": 103722,
    "Cook South": 103723,
    "Cottonwood": 103724,
    "Crow Wing": 103725,
    "Dakota": 103726,
    "Dodge": 103727,
    "Douglas": 103728,
    "Faribault": 103729,
    "Fillmore": 103730,
    "Freeborn": 103731,
    "Goodhue": 103732,
    "Grant": 103733,
    "Hennepin": 103734,
    "Houston": 1037235,
    "Hubbard": 103703,
    "Isanti": 103736,
    "Itasca North": 103737,
    "Itasca South": 103738,
    "Jackson": 103739,
    "Kanabec": 103740,
    "Kandiyohi": 103741,
    "Kittson": 103742,
    "Koochiching": 103743,
    "Lac qui Parle": 103744,
    "Lake": 103704,
    "Lake of the Woods North": 103745,
    "Lake of the Woods South": 103746,
    "Le Sueur": 103747,
    "Lincoln": 103748,
    "Lyon": 103749,
    "Mahnomen": 103751,
    "Marshall": 103752,
    "Martin": 103753,
    "McLeod": 103750,
    "Meeker": 103754,
    "Mille Lacs": 103705,
    "Morrison": 103755,
    "Mower": 103756,
    "Murray": 103757,
    "Nicollet": 103758,
    "Nobles": 103759,
    "Norman": 103760,
    "Olmsted": 103761,
    "Ottertail": 103762,
    "Pennington": 103763,
    "Pine": 103764,
    "Pipestone": 103765,
    "Polk": 103766,
    "Pope": 103767,
    "Ramsey": 103768,
    "Red Lake": 103769,
    "Redwood": 103770,
    "Renville": 103771,
    "Rice": 103772,
    "Rock": 103773,
    "Roseau": 103774,
    "St. Louis": 103695,
    "St. Louis Central": 103776,
    "St. Louis North": 103775,
    "St. Louis South": 103777,
    "Scott": 103778,
    "Sherburne": 103779,
    "Sibley": 103780,
    "Stearns": 103781,
    "Steele": 103782,
    "Stevens": 103783,
    "Swift": 103784,
    "Todd": 103785,
    "Traverse": 103786,
    "Wabasha": 103787,
    "Wadena": 103788,
    "Waseca": 103789,
    "Washington": 103706,
    "Watonwan": 103790,
    "Wilkin": 103707,
    "Winona": 103791,
    "Wright": 103792,
    "Yellow Medicine": 103793
}

reclass_rules = [
    "-5000 0 -1",
    "0 0.041000 1",
    "0.041000 0.083000 2",
    "0.083000 0.167000 3",
    "0.167000 0.250000 4",
    "0.250000 0.333000 5",
    "0.333000 0.417000 6",
    "0.417000 0.500000 7",
    "0.500000 0.583000 8",
    "0.583000 0.666000 9",
    "0.666000 0.750000 10",
    "0.750000 0.833000 11",
    "0.833000 0.916000 12",
    "0.916000 1.000000 13",
    "1.000000 5000 14"
] # Corresponds to how many rows are used in the remap table

epsg_code = mn_county_systems_epsg.get(input_county_name) # Use the input county name to get the WKID/ESPG
spatial_ref = arcpy.SpatialReference(epsg_code) # Create a Spatial Reference item with this projection WKID/ESPG
arcpy.management.DefineProjection(input_multipatch_path, spatial_ref) # Run the 'Define Projection' tool
  

cell_size = 0.2
cell_assignment_method="MAXIMUM_HEIGHT"
out_raster = f'Multipatch_to_Raster_Output_{input_project_name}'
arcpy.conversion.MultipatchToRaster(input_multipatch_path, out_raster, cell_size, cell_assignment_method) # Run the 'Multipatch to Raster' tool
print(f"Conversion Complete: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


reclass_field = "Value" # Field that is getting reclassed, will always just be value unless it's a multiband raster
remap = ";".join(reclass_rules[:input_numbers_rows + 1]) # Create a remap table object from the selected number of rows.
out_classify_raster = f'Reclassify_Output_{input_project_name}'
missing_values = "DATA" # Leave input NODATA values and NODATA
arcpy.ddd.Reclassify(out_raster, reclass_field, remap, out_classify_raster, missing_values) # Run the 'Reclassify' tool
print(f"ReClass Complete: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


out_polygon_features = f'Raster_Polygons_{input_project_name}' # Tool parameter
simplify = 'NO_SIMPLIFY' # Tool parameter
raster_field = 'Value' # Tool parameter
create_multipart_features = 'MULTIPLE_OUTER_PART' # Tool parameter
arcpy.conversion.RasterToPolygon(out_classify_raster, out_polygon_features, simplify, raster_field, create_multipart_features) # Run the 'Raster to Polygon' tool
print(f"Raster Conversion Complete: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


arcpy.management.AddField(out_polygon_features, 'milling_depth', 'TEXT') # Add the milling depth field
arcpy.management.AddField(out_polygon_features, 'area_sq_yd', 'DOUBLE') # Add the area in sq yards field


geometry_property = [['area_sq_yd', 'AREA']]
length_unit = ''
area_unit = 'SQUARE_YARDS_US'
arcpy.management.CalculateGeometryAttributes(out_polygon_features, geometry_property, length_unit, area_unit) # Calculate the area of each polygon in square yards

with arcpy.da.UpdateCursor(out_polygon_features, ['gridcode', 'milling_depth']) as cursor: # Script to add info to text field
    for row in cursor:
        gridcode = row[0]
        if gridcode == -1:
            row[1] = "Area above 0 inches"
        elif gridcode == 1:
            row[1] = "0 - 0.5 in."
        elif gridcode == 2:
            row[1] = "0.5 - 1 in."
        elif 3 <= gridcode < input_numbers_rows:
            row[1] = f"{gridcode - 2} - {gridcode - 1} in."
        elif gridcode == input_numbers_rows:
            row[1] = f"{gridcode - 2} in. or more"
        else:
            row[1] = "Outside of Multipatch"
        cursor.updateRow(row)  

        
arcpy.conversion.ExportTable(out_polygon_features, csv_save_location)

Started: 2024-07-25 12:33:08
Conversion Complete: 2024-07-25 12:38:18
ReClass Complete: 2024-07-25 12:42:44
Raster Conversion Complete: 2024-07-25 12:44:08


<Result 'C:\\Users\\lschneider\\OneDrive - Isthmus Engineering\\Documents\\Variable Milling TH 23.csv'>